In [21]:
# Importing libraries
import numpy as np
import pandas as pd
import cantera as ct
import matplotlib.pyplot as plt
import json

# Importing packages
import feedstock as fs
import pp
import gasifier as g
import fuel as fu
import energy as en
import outputs as op

In [22]:
# Grabbing data from JSON file
with open('data/data.json','r') as dataFile:
    data = json.load(dataFile)
    dataFile.close()

In [23]:
# Table2 = data['Thamavithya2010']['Table2']
# df = pd.DataFrame(Table2)
# df.head()

In [24]:
# fuelID = "ThamaSawdust"
# species=['CO','H2','CH4','CO2','N2']
# moist = fu.moisture(fuelID)
# n = len(Table2)
# ER = np.array([run['ER'] for run in Table2])
# T = np.array([run['Tgas'] for run in Table2]) + 273.15 # K
# experimental = np.zeros((n, len(species)))
# for i in range(n):
#     for j, sp in enumerate(species):
#         experimental[i,j] = Table2[i][sp]/100
# CC = np.array([run['CarbonConv'] for run in Table2])

In [25]:
# predicted = np.zeros_like(experimental)
# for i in range(n):
#     report0 = g.gasifier(fuelID, moist=moist, T=T[i], air=ER[i], airType='ER', species=species)
#     CC_pred = report0['CC']
#     C_avail0 = CC[i]/CC_pred
#     DT, sqerr = g.findTquasi(fuelID, experimental[i], moist=moist, T0=T[i], air=ER[i], airType='ER', C_avail=1.0, species=species)
#     report1 = g.gasifier(fuelID, moist=moist, T=T[i]-DT, air=ER[i], airType='ER', species=species+['C(gr)'], C_avail=1.0)
#     for j, sp in enumerate(species):
#         predicted[i,j] = report1[sp]
#     print(i)
# print(predicted)

In [26]:
# plt.scatter(ER, experimental[:,4], label='CO')
# plt.plot(ER, predicted[:,4], label='CO pred')
# plt.grid()
# plt.show()

In [27]:
# This uses data from 10.1007/s11708-020-0710-3 (Guo et al., 2020)

# Fuel is already in database (Compressed Corn Straw)
fuelID = 'GuoCorn'

GuoDict = data["Guo2021"]["Table3"]

# Experimental data information
T = np.array([700, 800, 900]) # °C
T = T + 273.15 # K
ER = 0.21 # constant
moist = 0
species = ['CO2', 'CO', 'CH4', 'H2']
experimental = np.array([[17.1, 14.7, 6.36, 7.58],
                         [16.5, 16.4, 6.07, 7.97],
                         [15.7, 17.8, 5.60, 8.35]]) # %
experimental = experimental/100 # fraction

In [28]:
# Creating a range of values for smooth, continuos model response
Tl = np.linspace(T[0], T[-1], 100) # K
CO2l = np.zeros_like(Tl)
COl = np.zeros_like(Tl)
CH4l = np.zeros_like(Tl)
H2l = np.zeros_like(Tl)

In [29]:
# # First test: raw model, no modifications

# for i, t in enumerate(Tl):
#     report = g.gasifier(fuelID, T=t, air=ER, species=species)
#     CO2l[i] = report['CO2']
#     COl[i] = report['CO']
#     CH4l[i] = report['CH4']
#     H2l[i] = report['H2']

# plt.plot(Tl, CO2l, label='CO2')
# plt.plot(Tl, COl, label='CO')
# plt.plot(Tl, CH4l, label='CH4') 
# plt.plot(Tl, H2l, label='H2')
# plt.scatter(T, experimental[:,0], label='CO2 exp')
# plt.scatter(T, experimental[:,1], label='CO exp')
# plt.scatter(T, experimental[:,2], label='CH4 exp')
# plt.scatter(T, experimental[:,3], label='H2 exp')
# plt.legend()
# plt.grid()
# plt.savefig("results.png")
# plt.show()

In [30]:
# # Second test: modified model finds DT but not C_avail, uses average DT value

# predicted = np.zeros_like(experimental)
# DTsum = 0
# for i, t in enumerate(T):
#     DT, sqerr = g.findDT(fuelID, experimental[i], T0=t, air=ER, species=species, C_avail=1.0)
#     report = g.gasifier(fuelID, T=t-DT, air=ER, species=species, C_avail=1.0)
#     for j, sp in enumerate(species):
#         predicted[i,j] = report[sp]
#     DTsum += DT
#     print(DT, sqerr)

# DTav = DTsum/len(T)
# print(DTav)

# for i, t in enumerate(Tl):
#     report = g.gasifier(fuelID, T=t-DTav, air=ER, species=species)
#     CO2l[i] = report['CO2']
#     COl[i] = report['CO']
#     CH4l[i] = report['CH4']
#     H2l[i] = report['H2']

# plt.plot(Tl, CO2l, label='CO2')
# plt.plot(Tl, COl, label='CO')
# plt.plot(Tl, CH4l, label='CH4') 
# plt.plot(Tl, H2l, label='H2')
# plt.scatter(T, experimental[:,0], label='CO2 exp')
# plt.scatter(T, experimental[:,1], label='CO exp')
# plt.scatter(T, experimental[:,2], label='CH4 exp')
# plt.scatter(T, experimental[:,3], label='H2 exp')
# plt.grid()
# plt.legend()
# plt.savefig("results2.png")
# plt.show()

In [31]:
# # Third test: modified model finds DT and C_avail, uses average values

# predicted = np.zeros_like(experimental)
# sums = [0, 0]
# for i, t in enumerate(T):
#     X = g.findParams(fuelID, experimental[i], T0=t, air=ER, species=species)
#     DT, C_avail, sqerr = X
#     report = g.gasifier(fuelID, T=t-DT, air=ER, species=species, C_avail=C_avail)
#     for j, sp in enumerate(species):
#         predicted[i,j] = report[sp]
#     sums[0] += DT
#     sums[1] += C_avail
#     print(DT, C_avail, sqerr)
# DTav = sums[0]/len(T)
# C_avail_av = sums[1]/len(T)
# print(DTav, C_avail_av)

# for i, t in enumerate(Tl):
#     report = g.gasifier(fuelID, T=t-DTav, air=ER, species=species, C_avail=C_avail_av)
#     CO2l[i] = report['CO2']
#     COl[i] = report['CO']
#     CH4l[i] = report['CH4']
#     H2l[i] = report['H2']

# plt.plot(Tl, CO2l, label='CO2')
# plt.plot(Tl, COl, label='CO')
# plt.plot(Tl, CH4l, label='CH4') 
# plt.plot(Tl, H2l, label='H2')
# plt.scatter(T, experimental[:,0], label='CO2 exp')
# plt.scatter(T, experimental[:,1], label='CO exp')
# plt.scatter(T, experimental[:,2], label='CH4 exp')
# plt.scatter(T, experimental[:,3], label='H2 exp')
# plt.grid()
# plt.legend()
# plt.savefig("results3.png")
# plt.show()

In [32]:
GuoDF = pd.DataFrame.from_dict(GuoDict)

n = len(GuoDict)
T = np.zeros(n)
ER = np.zeros(n)
SB = np.zeros(n)
DMP = np.zeros(n)
species = ['CO2', 'CO', 'CH4', 'H2']
experimental = np.zeros((n, len(species)))
for i, run in enumerate(GuoDict):
    T[i] = run['T']
    ER[i] = run['ER']
    SB[i] = run['S/B']
    DMP[i] = run['DMP']
    for j, sp in enumerate(species):
        experimental[i,j] = run[sp]/100
GuoDF.head()

,Run,T,ER,S/B,DMP,CO2,CO,CH4,H2,Tar
0,1,700,0.21,0.0,0,17.1,14.7,6.36,7.58,6.20
1,2,740,0.26,0.0,0,17.8,15.5,7.24,7.93,5.56
2,3,770,0.31,0.0,0,18.8,16.4,7.89,8.91,4.69
3,4,820,0.34,0.0,0,20.2,15.2,6.41,8.07,3.97
4,5,700,0.21,0.2,0,17.8,12.9,4.21,9.51,5.34


In [33]:
predicted = np.zeros_like(experimental)
for i, run in enumerate(GuoDict):
    report0 = g.gasifier(fuelID, T=T[i], air=ER[i], airType='ER', stm=SB[i], stmType='steam', species=species, C_avail=1.0)
    sqErr = 0
    for j, sp in enumerate(species):
        predicted[i,j] = report0[sp]
        sqErr += (predicted[i,j]*100 - experimental[i,j]*100)**2
    RMSE = np.sqrt(sqErr/len(species)) # as a percentage
    print(i, RMSE)

0 7.112904790932172
1 7.347066418825537
2 7.516538883118779
3 6.105036547508926
4 6.237958749675787
5 6.315572303190121
6 6.330903897473092
7 7.572430461090417
8 8.448768618916969
9 8.861960467922781
10 7.396938515312028
11 7.453267980941122


In [34]:
predicted = np.zeros_like(experimental)
for i, run in enumerate(GuoDict):
    DT, sqerr = g.findDT(fuelID, experimental[i], T0=T[i], air=ER[i], airType='ER', stm=SB[i], stmType='steam', species=species, C_avail=1.0)
    report1 = g.gasifier(fuelID, T=T[i]-DT, air=ER[i], airType='ER', stm=SB[i], stmType='steam', species=species, C_avail=1.0)
    sqErr = 0
    for j, sp in enumerate(species):
        predicted[i,j] = report1[sp]
        sqErr += (predicted[i,j]*100 - experimental[i,j]*100)**2
    RMSE = np.sqrt(sqErr/len(species)) # as a percentage
    print(i, DT, RMSE)

0 -111.09667743222224 6.815220163839558
1 -115.57561846303884 6.497014255712355
2 -103.25736180955917 6.124447513928217
3 -41.461838709634115 5.744900663536813
4 -60.68077360350815 5.907703903210678
5 -91.01303506201175 4.992208519293353
6 -88.15130878433142 4.505880568755414
7 -145.19455462328114 6.74303781992107
8 -169.37428423863372 6.485726249331382
9 -175.9004209664357 6.212227975618635
10 -53.99391467653657 7.1528484547585975
11 25.204537412866028 7.261026474506891
